In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import opttrot


## Fast Pauli decomposition algorithm implementation

```
Hantzko, Binkowski, and Gupta, Tensorized Pauli decomposition algorithm
```

In [3]:
import numpy as np
from copy import deepcopy

In [4]:
from opttrot import utils 

In [55]:
# Size = 2^n Hermit matrix
n =3
size_n = int(2**n)
A = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
ImA = np.asmatrix(np.random.rand(size_n, size_n).astype(complex))
A = A.H@A
ImA = ImA.H@ImA
H = A+1j*ImA

In [56]:
pauli_matrix, pauli_terms = opttrot.Hamiltonian.generate_pauli_terms(n) # Get 2^n pauli-terms

In [57]:
# Frobenius inner product
decomposition = {}
for matrix, term in zip(pauli_matrix, pauli_terms):
    coef = utils.frobenius_inner(matrix, H)
    decomposition[term] = coef

In [58]:
decomposition

{'III': (2.40640839661886+2.1423272063545706j),
 'IIZ': (0.1208985770308244+0.25431370257375024j),
 'IZI': (-0.16737812372628325-0.12174086376292226j),
 'IZZ': (-0.23933556468842476-0.3046000214238696j),
 'ZII': (0.10769945523644359-0.17593902357749275j),
 'ZIZ': (-0.26741620807943184+0.09918923748402803j),
 'ZZI': (0.20003160004891346-0.06569710023532904j),
 'ZZZ': (-0.07172214401935012+0.07627223384781034j),
 'IIX': (1.6087834631278974+1.6480481965939713j),
 'IIY': 0j,
 'IZX': (-0.22856382510350826-0.2503372530168946j),
 'IZY': 0j,
 'ZIX': (0.011549165833156932-0.16758330679488642j),
 'ZIY': 0j,
 'ZZX': (0.38678859302994506-0.02289270761512685j),
 'ZZY': 0j,
 'IXI': (1.9222499110377442+1.6268944223768693j),
 'IXZ': (-0.08069696633930823+0.20717214884591018j),
 'IYI': 0j,
 'IYZ': 0j,
 'ZXI': (0.26289937096845495-0.2669973970686793j),
 'ZXZ': (-0.12398834952167037+0.31181204913800886j),
 'ZYI': 0j,
 'ZYZ': 0j,
 'IXX': (1.737031562412017+1.5219549175261866j),
 'IXY': 0j,
 'IYX': 0j,
 'I

In [62]:
H_test = deepcopy(H)

### Implementation

In [63]:
exp2n = H_test.shape[0]
n = int(np.log2(exp2n))

print("Hamiltonian shape:", H_test.shape)
print("Total qubits:", n)


Hamiltonian shape: (8, 8)
Total qubits: 3


In [64]:
# Implementation of decomposition
l = int(exp2n)
for i in range(n): #range(n)
    m = int(2**i)
    l = int(l/2)

    print("number of submatrix:", m)
    print("sub matrix unit:", l)

    for j in range(m):
        for k in range(m):

            num_i = j*l
            num_j = k*l

            print(f"{num_i}, {num_j}")

            print(f"I-Z: {num_i}:{num_i+l}, {num_j}:{num_j+l} | {num_i+l}: {num_i+2*l}, {num_j+l}:{num_j+2*l}")
            H_test[num_i: num_i+l, num_j:num_j+l] += H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
            H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H_test[num_i: num_i+l, num_j:num_j+l] - 2*H_test[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

            print(f"X-Y: {num_i}:{num_i+l}, {num_j+l}:{num_j+2*l} | {num_i+l}: {num_i+2*l}, {num_j}:{num_j+l}")
            H_test[num_i: num_i+l, num_j+l:num_j+2*l] += H_test[num_i+l: num_i+2*l, num_j:num_j+l] 
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] =  H_test[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H_test[num_i+l: num_i+2*l, num_j:num_j+l]
            H_test[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j
    print("------------------------------------------------")
    H_test *= 0.5


number of submatrix: 1
sub matrix unit: 4
0, 0
I-Z: 0:4, 0:4 | 4: 8, 4:8
X-Y: 0:4, 4:8 | 4: 8, 0:4
------------------------------------------------
number of submatrix: 2
sub matrix unit: 2
0, 0
I-Z: 0:2, 0:2 | 2: 4, 2:4
X-Y: 0:2, 2:4 | 2: 4, 0:2
0, 2
I-Z: 0:2, 2:4 | 2: 4, 4:6
X-Y: 0:2, 4:6 | 2: 4, 2:4
2, 0
I-Z: 2:4, 0:2 | 4: 6, 2:4
X-Y: 2:4, 2:4 | 4: 6, 0:2
2, 2
I-Z: 2:4, 2:4 | 4: 6, 4:6
X-Y: 2:4, 4:6 | 4: 6, 2:4
------------------------------------------------
number of submatrix: 4
sub matrix unit: 1
0, 0
I-Z: 0:1, 0:1 | 1: 2, 1:2
X-Y: 0:1, 1:2 | 1: 2, 0:1
0, 1
I-Z: 0:1, 1:2 | 1: 2, 2:3
X-Y: 0:1, 2:3 | 1: 2, 1:2
0, 2
I-Z: 0:1, 2:3 | 1: 2, 3:4
X-Y: 0:1, 3:4 | 1: 2, 2:3
0, 3
I-Z: 0:1, 3:4 | 1: 2, 4:5
X-Y: 0:1, 4:5 | 1: 2, 3:4
1, 0
I-Z: 1:2, 0:1 | 2: 3, 1:2
X-Y: 1:2, 1:2 | 2: 3, 0:1
1, 1
I-Z: 1:2, 1:2 | 2: 3, 2:3
X-Y: 1:2, 2:3 | 2: 3, 1:2
1, 2
I-Z: 1:2, 2:3 | 2: 3, 3:4
X-Y: 1:2, 3:4 | 2: 3, 2:3
1, 3
I-Z: 1:2, 3:4 | 2: 3, 4:5
X-Y: 1:2, 4:5 | 2: 3, 3:4
2, 0
I-Z: 2:3, 0:1 | 3: 4, 1:2
X-Y:

In [35]:
decomposition

{'II': (0.9005362940313093+1.3204313675811121j),
 'IZ': (-0.08622551788374488-0.08438509874824277j),
 'ZI': (-0.37607936649825796-0.2072217820735276j),
 'ZZ': (-0.2333821608383625+0.30023520183510644j),
 'IX': (0.7635570833803782+0.9990637222435708j),
 'IY': 0j,
 'ZX': (-0.3685600234469523-0.32519660297994957j),
 'ZY': 0j,
 'XI': (0.4870871213098663+0.9587796614603952j),
 'XZ': (-0.061228390080900724-0.00888796892719318j),
 'YI': 0j,
 'YZ': 0j,
 'XX': (0.521002575605159+1.058572172370484j),
 'XY': 0j,
 'YX': 0j,
 'YY': (0.24851833341979765-0.17550013528666797j)}

In [49]:
m11 = H_test[:2,:2]
m12 = H_test[:2,2:] 
m21 = H_test[2:,:2]
m22 = H_test[2:,2:]

In [50]:
coef_list = []

In [51]:
coef_list.append((m11[0,0] + m11[1,1])/2)
coef_list.append((m11[0,0] - m11[1,1])/2)
coef_list.append((m11[0,1] + m11[1,0])/2)
coef_list.append((m11[0,1] - m11[1,0])*(-j)/2)
coef_list.append((m12[0,0] + m12[1,1])/2)
coef_list.append((m12[0,0] - m12[1,1])/2)
coef_list.append((m12[0,1] + m12[1,0])/2)
coef_list.append((m12[0,1] - m12[1,0])*(-j)/2)
coef_list.append((m21[0,0] + m21[1,1])/2)
coef_list.append((m21[0,0] - m21[1,1])/2)
coef_list.append((m21[0,1] + m21[1,0])/2)
coef_list.append((m21[0,1] - m21[1,0])*(-j)/2)
coef_list.append((m22[0,0] + m22[1,1])/2)
coef_list.append((m22[0,0] - m22[1,1])/2)
coef_list.append((m22[0,1] + m22[1,0])/2)
coef_list.append((m22[0,1] - m22[1,0])*(-j)/2)

In [52]:
coef_list

[(0.9005362940313093+1.3204313675811121j),
 (-0.08622551788374494-0.08438509874824285j),
 (0.7635570833803782+0.9990637222435708j),
 0j,
 (0.4870871213098663+0.9587796614603952j),
 (-0.061228390080900724-0.00888796892719318j),
 (0.521002575605159+1.058572172370484j),
 0j,
 0j,
 0j,
 (1.1102230246251565e-16+2.7755575615628914e-17j),
 0j,
 (-0.37607936649825796-0.2072217820735277j),
 (-0.23338216083836255+0.3002352018351063j),
 (-0.3685600234469524-0.32519660297994957j),
 0j]

(0.7635570833803782+0.9990637222435708j)

-0j

In [36]:
from pd import PauliDecomposition

In [37]:
pstring, pcoef = PauliDecomposition(H)
pdecompose = {st:coef for st, coef in zip(pstring, pcoef)}

In [38]:
pdecompose

{'II': (0.9005362940313093+1.3204313675811121j),
 'IX': (0.7635570833803782+0.9990637222435708j),
 'IZ': (-0.08622551788374494-0.08438509874824285j),
 'XI': (0.4870871213098663+0.9587796614603952j),
 'XX': (0.521002575605159+1.058572172370484j),
 'XZ': (-0.061228390080900724-0.00888796892719318j),
 'YY': (0.24851833341979765-0.17550013528666797j),
 'ZI': (-0.376079366498258-0.20722178207352765j),
 'ZX': (-0.3685600234469523-0.32519660297994957j),
 'ZZ': (-0.23338216083836252+0.3002352018351064j)}

In [14]:
def fast_p_decompose(H):
    exp2n = H.shape[0]
    n = int(np.log2(exp2n))
    # Implementation of decomposition
    l = int(exp2n)
    for i in range(n):
        m = int(2**i)
        l = int(l/2)
        for j in range(m):
            for k in range(m):

                num_i = j*l
                num_j = k*l

                H[num_i: num_i+l, num_j:num_j+l] += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]  = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]

                H[num_i: num_i+l, num_j+l:num_j+2*l] += H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l] =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= -1j

        H *= 0.5
    return H

# string to matrix

In [17]:
from opttrot.utils import pstr_to_matrix, pstr_to_xz_fam_code

In [18]:
pstr= "IZX"
xz_code = pstr_to_xz_fam_code(pstr)
p_matrix = pstr_to_matrix(pstr)

In [19]:
p_matrix

array([[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -0.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j,
         0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -0.+0.j,
        -1.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j,
        -0.+0.j]])

In [20]:
xz_code

(1, 2)

In [26]:
ma = fast_p_decompose(p_matrix)
nx, ny = ma.shape

for i in range(nx):
    for j in range(ny):
        print(ma[i][j], " ", end="")
    print("")

0j  (-0.0380859375+0.0517578125j)  0j  (0.0126953125-0.01953125j)  0j  (0.0498046875-0.0126953125j)  0j  (0.0244140625+0.021484375j)  
(0.1259765625-0.048828125j)  0j  (0.0234375-0.0703125j)  0j  (0.0126953125-0.05078125j)  0j  (0.0029296875+0.017578125j)  0j  
0j  (0.1201171875-0.0126953125j)  0j  (0.1494140625-0.0078125j)  0j  (0.0234375+0.021484375j)  0j  0j  
(0.091796875+0.060546875j)  0j  (0.15234375-0.1865234375j)  0j  (0.1728515625-0.2529296875j)  0j  0j  0j  
0j  (0.0107421875-0.056640625j)  0j  (-0.0263671875-0.0615234375j)  0j  (-0.01953125+0.0009765625j)  0j  (-0.00390625-0.01171875j)  
(0.013671875+0.0048828125j)  0j  (0.033203125+0.017578125j)  0j  (-0.0185546875-0.0029296875j)  0j  (0.01953125-0.0205078125j)  0j  
0j  (0.00390625-0.0107421875j)  0j  0j  0j  (0.0048828125+0.01953125j)  0j  (-0.0234375-0.0048828125j)  
(-0.0146484375-0.04296875j)  0j  0j  0j  (-0.0224609375+0.048828125j)  0j  (-0.00390625+0.021484375j)  0j  
